# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import main libraries 
import pandas as pd
import numpy as np

#import text libraries
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('wordnet') # download for lemmatization
from sqlalchemy import create_engine
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.datasets import make_multilabel_classification
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [32]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("InsertTableName", engine)
X = df['message'].values
Y = df.drop(['message','original', 'id', 'genre'], axis=1)

In [41]:
print(X.shape)
print(Y.shape)

(26216,)
(26216, 36)


In [42]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 36 columns):
related                   26216 non-null int64
request                   26216 non-null int64
offer                     26216 non-null int64
aid_related               26216 non-null int64
medical_help              26216 non-null int64
medical_products          26216 non-null int64
search_and_rescue         26216 non-null int64
security                  26216 non-null int64
military                  26216 non-null int64
child_alone               26216 non-null int64
water                     26216 non-null int64
food                      26216 non-null int64
shelter                   26216 non-null int64
clothing                  26216 non-null int64
money                     26216 non-null int64
missing_people            26216 non-null int64
refugees                  26216 non-null int64
death                     26216 non-null int64
other_aid                 26216 non-null int6

In [60]:
category_names = Y.columns
cat_names = []

In [63]:
list(cat_names)

[array(['related'], 
       dtype='<U32'), array(['request'], 
       dtype='<U32'), array(['offer'], 
       dtype='<U32'), array(['aid_related'], 
       dtype='<U32'), array(['medical_help'], 
       dtype='<U32'), array(['medical_products'], 
       dtype='<U32'), array(['search_and_rescue'], 
       dtype='<U32'), array(['security'], 
       dtype='<U32'), array(['military'], 
       dtype='<U32'), array(['child_alone'], 
       dtype='<U32'), array(['water'], 
       dtype='<U32'), array(['food'], 
       dtype='<U32'), array(['shelter'], 
       dtype='<U32'), array(['clothing'], 
       dtype='<U32'), array(['money'], 
       dtype='<U32'), array(['missing_people'], 
       dtype='<U32'), array(['refugees'], 
       dtype='<U32'), array(['death'], 
       dtype='<U32'), array(['other_aid'], 
       dtype='<U32'), array(['infrastructure_related'], 
       dtype='<U32'), array(['transport'], 
       dtype='<U32'), array(['buildings'], 
       dtype='<U32'), array(['electricity'],

In [61]:
cat_names = [np.append(cat_names, a) for a in Y.columns]

In [25]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    # ELIMINAMOS LOS ESPACIOS, TRANSFORMAMOS EN LOWERCASE
    #utilizamos el paquete re para eliminar todo lo que no sea [^a-zA-Z0-9]
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()) 
    #tokenizamos el texto
    tokens = text.split()
    #eliminamos aquellos tokens que son stopwords
    words = [w for w in tokens if w not in stopwords.words("english")]
    #transformamos las palabras en su raíz, para evitar derivaciones
    lemmed_tokens = [WordNetLemmatizer().lemmatize(w) for w in words]
    return lemmed_tokens

In [7]:
# tokenize(X[0])

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
from sklearn.pipeline import Pipeline, FeatureUnion

pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42)))
     
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [35]:
x_train,x_test,y_train,y_test= train_test_split(X,Y, test_size=0.33, random_state=42)

pipeline.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=1))])

In [83]:
y_pred.to_list()

AttributeError: 'numpy.ndarray' object has no attribute 'to_list'

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [36]:
y_pred = pipeline.predict(x_test)

In [78]:
results = classification_report(y_test.values, y_pred, target_names = Y.columns)

ValueError: Unknown label type: (array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 1, 1, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]]), array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]]))

In [55]:
#show the results of first pipeline model
columns = Y.columns
[print(columns[count],classification_report(y_test.values[:,count], y_pred[:,count]), sep="\n") for count in range(len(y_pred[0,:]))]

related
             precision    recall  f1-score   support

          0       0.62      0.46      0.53      2054
          1       0.84      0.91      0.87      6534
          2       0.28      0.25      0.26        64

avg / total       0.78      0.80      0.79      8652

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      7180
          1       0.79      0.43      0.56      1472

avg / total       0.88      0.88      0.87      8652

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8614
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      8652

aid_related
             precision    recall  f1-score   support

          0       0.76      0.84      0.80      5107
          1       0.73      0.61      0.66      3545

avg / total       0.75      0.75      0.74      8652

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [57]:
y_pred

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f6148d78598>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=42, v

In [12]:
parameters = {
    'clf__estimator__criterion': ['gini','entropy'],
    'clf__estimator__max_features': ['auto','sqrt'],
    'clf__estimator__n_jobs': [-1],
    'clf__estimator__verbose': [2],
}

cv = GridSearchCV(pipeline, param_grid = parameters, n_jobs = 12)

In [52]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
cv.fit(x_train,y_train)
y_pred_cv = cv.predict(x_test)

building tree 1 of 10
building tree 1 of 10
building tree 1 of 10
building tree 1 of 10
building tree 1 of 10
building tree 1 of 10
building tree 1 of 10


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    2.9s remaining:    0.0s


building tree 2 of 10


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    3.0s remaining:    0.0s


building tree 2 of 10
building tree 1 of 10


...

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.9s finished
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10
building tree 1 of 10


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.6s finished
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    3.5s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.1s finished
[Parallel(n_jobs=1)]: Done   

In [21]:
cv_best_estimator = cv.best_estimator_

In [23]:
cv.best_params_

{'clf__estimator__criterion': 'gini',
 'clf__estimator__max_features': 'auto',
 'clf__estimator__n_jobs': -1,
 'clf__estimator__verbose': 2}

In [24]:
[print(columns[count],classification_report(y_test[:,count], y_pred_cv[:,count]), sep="\n") for count in range(len(y_pred_cv[0,:]))]

related
             precision    recall  f1-score   support

          0       0.62      0.46      0.53      2054
          1       0.84      0.91      0.87      6534
          2       0.28      0.25      0.26        64

avg / total       0.78      0.80      0.79      8652

request
             precision    recall  f1-score   support

          0       0.89      0.98      0.93      7180
          1       0.79      0.43      0.56      1472

avg / total       0.88      0.88      0.87      8652

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8614
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      8652

aid_related
             precision    recall  f1-score   support

          0       0.76      0.84      0.80      5107
          1       0.73      0.61      0.66      3545

avg / total       0.75      0.75      0.74      8652

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
#creamos un estimador que cambia al estimador que queramos
# https://stackoverflow.com/questions/50285973/pipeline-multiple-classifiers
from sklearn.base import BaseEstimator
from sklearn.linear_model import SGDClassifier

class ClfSwitcher(BaseEstimator):

    def __init__(
        self, 
        estimator = DecisionTreeClassifier(),
    ):
        """
        A Custom BaseEstimator that can switch between classifiers.
        :param estimator: sklearn object - The classifier
        """ 

        self.estimator = estimator


    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self


    def predict(self, X, y=None):
        return self.estimator.predict(X)


    def predict_proba(self, X):
        return self.estimator.predict_proba(X)


    def score(self, X, y):
        return self.estimator.score(X, y)
    


In [13]:
# https://stackoverflow.com/questions/50285973/pipeline-multiple-classifiers

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.naive_bayes import MultinomialNB


pipeline_2 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator = ClfSwitcher())),
])

# parameters_2 = {{
#         'clf__estimator': DecisionTreeClassifier(),
#         'clf__estimator__estimator__criterion': ['gini', 'entropy'],
#         'clf__estimator__estimator__max_features': ['sqrt', 'log2'],
#         'clf__estimator__estimator__splitter': ['best','random'],
#         'clf__estimator__estimator__class_weight': [None,'balanced'],
#         'clf__estimator__estimator__max_depth':[ None, 5,8,10],
#     },

#     {
#         'clf__estimator': [SVC()], 
#         'clf_estimator__estimator_gamma':['scale', 'auto'],
#         'clf__estimator__estimator_kernel': ['linear', 'rbf'],
#         'clf__estimator__C':[1,10]
#     }
#     }





In [14]:
rd_cv = RandomizedSearchCV(pipeline_2, param_distributions = parameters_2, n_jobs = 12)
rd_cv.fit(x_train,y_train)

NameError: name 'parameters_2' is not defined

In [81]:
pipeline_2.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f042707f730>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=ClfSwitcher(estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, presort=False, random_state=None,
               splitter='best')),
     

In [15]:
pipeline_3 = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(estimator = DecisionTreeClassifier())),
])

parameters_3 = {
        'clf__estimator__criterion': ['gini', 'entropy'],
        'clf__estimator__max_features': ['sqrt', 'log2'],
        'clf__estimator__splitter': ['best','random'],
        'clf__estimator__class_weight': [None,'balanced'],
        'clf__estimator__max_depth':[ None, 5,8,10]
    }


In [16]:
rd_cv_2 = RandomizedSearchCV(pipeline_3, param_distributions = parameters_3, n_jobs = 12)
rd_cv_2.fit(x_train,y_train)

RandomizedSearchCV(cv=None, error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ion_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
           n_jobs=1))]),
          fit_params=None, iid=True, n_iter=10, n_jobs=12,
          param_distributions={'clf__estimator__criterion': ['gini', 'entropy'], 'clf__estimator__max_features': ['sqrt', 'log2'], 'clf__estimator__splitter': ['best', 'random'], 'clf__estimator__class_weight': [None, 'balanced'], 'clf__estimator__max_depth': [None, 5, 8, 10]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [18]:
rd_cv_2_best_estimator = rd_cv_2.best_estimator_

In [23]:
y_pred_rd_cv = rd_cv_2_best_estimator.predict(x_test)

In [24]:
[print(columns[count],classification_report(y_test[:,count], y_pred_rd_cv[:,count]), sep="\n") for count in range(len(y_pred_rd_cv[0,:]))]

related
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      2054
          1       0.76      1.00      0.86      6534
          2       0.00      0.00      0.00        64

avg / total       0.57      0.76      0.65      8652

request
             precision    recall  f1-score   support

          0       0.83      1.00      0.91      7180
          1       0.50      0.00      0.01      1472

avg / total       0.77      0.83      0.75      8652

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      8614
          1       0.00      0.00      0.00        38

avg / total       0.99      1.00      0.99      8652

aid_related
             precision    recall  f1-score   support

          0       0.59      0.98      0.74      5107
          1       0.49      0.03      0.05      3545

avg / total       0.55      0.59      0.46      8652

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### 9. Export your model as a pickle file

In [3]:
import pickle

In [27]:
# https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/

# Save the Modle to file in the current working directory

Pkl_Filename = "Cv_best_model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(cv_best_estimator, file)

In [22]:
with open('rd_cv_2_best_estimator.pkl', 'wb') as file:  
    pickle.dump(rd_cv_2_best_estimator, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [7]:
model = pickle.load(open("Cv_best_model.pkl", 'rb'))

In [8]:
model

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...,
            oob_score=False, random_state=42, verbose=2, warm_start=False),
           n_jobs=1))])